In [1]:
#importing packages
import pandas as pd
from bs4 import BeautifulSoup
import requests
import string
import sys
import concurrent.futures
import time


### Downloading body text

In [2]:
t1 = time.perf_counter()
final = {}

def download_text(url):
    try:
        global info
        info = {}
        r = requests.get(url, headers = headers)
        time.sleep(10)
        html_doc= r.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        #print(soup.prettify())
        if soup.find_all('p'):
            y =""
            for x in soup.find_all('p'):
                y += x.get_text()
            info[soup.title.text] = y
        elif soup.find_all('section'):
            y = ""
            for x in soup.find_all('section'): #section class = 'articlebody'
                y += x.get_text()
            info[soup.title.text] = y
    except:
        e = sys.exc_info()[0]
        print(e)
        print(url)
    
    final.update(info)
######################Experimental    
  
for chunk in pd.read_csv("compiled_articles.csv",chunksize = 50):   #INPUT CSV FILE HERE 
#######################################
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(download_text, chunk.iloc[:,5])
    
t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')

#Final ouput is dictionary called 'final'

In [ ]:
#confirmation that we have all/most the articles
i = pd.read_csv('')
print('The length of the original dataset is ' , len(i))
print("\n")
print('The numer of articles we succesfully grabbed body text from is ', len(final))

In [ ]:
#dictionary to dataframe
df = pd.DataFrame(list(final.items()),columns = ['title','body_text']) 

In [ ]:
#dataframe to csv
df.to_csv("Articles_text.csv",index = False)